## [분야1] 완전 자율 네트워크 운영을 위한 초단기 네트워크 트래픽 예측 베이스라인 코드
#### 주의: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.

### 추론 실행 환경
1. python 3.10 환경
2. torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
    - 최신 버전입니다.

코드는 크게 3가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 및 추가 필요 라이브러리 설치
    - 채점 및 제출을 위한 aifactory 라이브러리를 설치하는 셀입니다. 이 부분은 수정하지 않고 그대로 실행합니다.
    - 필요 라이브러리를 직접 설치합니다.
2. 추론용 코드 작성
    - 모델 로드, 데이터 전처리, 예측 등 실제 추론을 수행하는 모든 코드를 이 영역에 작성합니다.
3. aif.submit() 함수를 호출하여 최종 결과를 제출하는 파트입니다.
    - 마이 페이지에서 발급받은 key 값을 함수의 인자로 정확히 입력해야 합니다.

※ 가능하면 제출시에는 포함되어 있는 train data를 폴더에서 제외하고 제출하시는 편이 좋습니다.
    - 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 시간 감소

### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [1]:
!pip install -U aifactory

  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Preparing metadata (setup.py) ... done
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 14.0 MB/s eta 0:00:00
  DEPRECATION: Building 'docopt' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'docopt'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13782 sha256=21691134bd57841fa4ca423ee4f8193e8ffbdb8eed90aac0b6a9b9e1c0479b8e
  Stored in directory: /Users/seolwootae/Library/Caches/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
  A

In [3]:
# 자신의 모델에 필요한 추가 라이브러리 설치
!pip install numpy pandas pytorch-lightning scikit-learn
!pip install torch torchvision torchaudio

In [4]:
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 kB 23.7 MB/s eta 0:00:00


In [6]:
!pip install joblib

### 2. 추론 환경의 기본 경로 구조
#### 제출 시 주의사항

1. 테스트 데이터 경로: /aif/data/test_inputs.pkl
   - 채점에 사용될 테스트 입력 데이터는 이 디렉토리 안에 test_inputs.pkl이라는 이름으로 고정되어 제공됩니다.
2. 모델 및 자원 경로: 예시 : ./model/
   - 추론 스크립트가 실행되는 위치를 기준으로, 제출된 모델 관련 파일들이 위치하는 상대 경로입니다.
   - 학습된 모델 가중치(.pt, .ckpt, .pth 등)
3. 제출 파일은 submission.pkl로 저장
4. argparse 사용시 args, _ = parser.parse_known_args()로 인자 지정
   args = parser.parse_args()는 jupyter에서 오류가 발생합니다!!!
5. return 할 결과물과 양식에 유의합니다.

In [5]:
import os
import numpy as np
import pandas as pd
import glob
import joblib
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import pickle
import json

# 0. 재현성을 위한 시드 고정
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 디바이스 설정 (GPU 우선 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEQ_LEN = 100  # 테스트 데이터와 맞춤
HIDDEN_SIZE = 128  # 더 긴 시퀀스를 위해 증가
BATCH_SIZE = 32
# 1. 테스트 데이터셋 클래스
class TrafficTestDataset(Dataset):
    def __init__(self, x_data, seq_len=10):
        self.data = x_data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.data[idx])

# 2. LSTM 모델 정의
class LSTMModel(pl.LightningModule):
    def __init__(self, input_size, hidden_size=HIDDEN_SIZE, num_layers=1, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # 마지막 시퀀스의 출력 사용
        last_output = lstm_out[:, -1, :]
        x = self.dropout(last_output)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x  # (batch_size, 1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

# 3. 저장된 모델과 스케일러 로드
def load_model_components(model_name="traffic_lstm_model",
                         model_path="./model"):
    # 메타데이터 로드
    with open(f'{model_path}/{model_name}_meta.json', 'r') as f:
        metadata = json.load(f)

    # X 스케일러 로드
    with open(f'{model_path}/{model_name}_scaler.pkl', 'rb') as f:
        x_scaler = pickle.load(f)

    # Y 스케일러 로드 (있다면)
    y_scaler_path = f'{model_path}/{model_name}_y_scaler.pkl'

    with open(y_scaler_path, 'rb') as f:
        y_scaler = pickle.load(f)


    # 모델 로드
    input_size = metadata.get('input_size', 27)
    hidden_size = metadata.get('hidden_size', 128)
    num_layers = metadata.get('num_layers', 1)
    dropout = metadata.get('dropout', 0.2)

    model = LSTMModel(input_size, hidden_size, num_layers, dropout)
    model.load_state_dict(torch.load(f'{model_path}/{model_name}.pth', map_location=device))
    model.to(device)
    model.eval()

    return model, x_scaler, y_scaler, metadata

# 4. 테스트 데이터 로드 및 전처리
test_data = joblib.load("/aif/data/test_inputs.pkl")

# 모델 컴포넌트 로드
model_name = "traffic_lstm_model"
model, x_scaler, y_scaler, metadata = load_model_components(model_name)

# 5. 테스트 데이터 전처리
processed_test_data = []

# test_data가 (163854, 100, 27) 형태의 numpy array인 경우
if isinstance(test_data, np.ndarray) and len(test_data.shape) == 3:
    # 효율적인 방법: 전체를 한번에 처리
    n_samples, original_seq_len, n_features = test_data.shape

    # 2D로 reshape하여 스케일링
    test_data_2d = test_data.reshape(-1, n_features)
    test_data_scaled_2d = x_scaler.transform(test_data_2d)
    test_data_scaled = test_data_scaled_2d.reshape(n_samples, original_seq_len, n_features)

    # 시퀀스 길이 조정 (SEQ_LEN에 맞춤)
    seq_len = metadata.get('seq_len', 10)

    for i in range(n_samples):
        sample = test_data_scaled[i]  # (100, 27)

        if seq_len <= original_seq_len:
            # 마지막 seq_len개만 사용
            sequence = sample[-seq_len:]
        else:
            # 패딩이 필요한 경우 (거의 없을 것)
            pad_len = seq_len - original_seq_len
            padding = np.zeros((pad_len, n_features))
            sequence = np.vstack([padding, sample])

        processed_test_data.append(sequence)

else:
    # test_data가 리스트 형태인 경우 (예전 방식)
    for sample in test_data:
        # 각 샘플을 개별적으로 처리
        sample_array = np.array(sample) if not isinstance(sample, np.ndarray) else sample

        # 스케일링 적용
        sample_scaled = x_scaler.transform(sample_array)

        # 시퀀스 길이 확인 및 조정
        seq_len = metadata.get('seq_len', 10)
        if len(sample_scaled) >= seq_len:
            sequence = sample_scaled[-seq_len:]
        else:
            pad_len = seq_len - len(sample_scaled)
            padding = np.zeros((pad_len, sample_scaled.shape[1]))
            sequence = np.vstack([padding, sample_scaled])

        processed_test_data.append(sequence)

# numpy array로 변환
processed_test_data = np.array(processed_test_data)

# 6. 데이터셋 및 데이터로더 생성
test_dataset = TrafficTestDataset(processed_test_data)
BATCH_SIZE = 256
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# 7. 추론 실행
y_pred = []

with torch.no_grad():
    test_pbar = tqdm(test_loader, desc="[base_line]")
    for x_batch in test_pbar:
        x_batch = x_batch.to(device)
        outputs = model(x_batch)  # (batch_size, 1)
        
        # peak_volume 예측값
        outputs_cpu = outputs.cpu().numpy().flatten()  # (batch_size,)
        
        # y_scaler가 있으면 역변환
        if y_scaler is not None:
            outputs_original = y_scaler.inverse_transform(outputs_cpu.reshape(-1, 1))
        else:
            outputs_original = outputs_cpu
        
        y_pred.append(outputs_original)

# 8. 결과 합치기
y_pred = np.concatenate(y_pred, axis=0)

# 9. 결과 저장 (numpy array 형태)
joblib.dump(y_pred, "submission.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '/aif/data/test_inputs.pkl'

### 3. 제출하기 
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 대회 페이지 [베이스라인 코드](https://aifactory.space/task/9162/baseline) 탭에 기재된 가이드라인을 따라 task별로 확인하실 수 있습니다.
- key가 틀리면 제출이 진행되지 않거나 잘못 제출되므로 태스크에 맞는 자기 key를 사용해야 합니다.

In [10]:
import aifactory.score as aif
import time
t = time.time()

#-----------------------------------------------------#
aif.submit(model_name="new_test",
           key="f432cb70-58cc-4748-841b-a1b7ba92d5fd"
           )
#-----------------------------------------------------#
print(time.time() - t)

file : task
jupyter notebook
이 대회는 <u>10분</u> 마다 제출이 가능합니다.
                    <br> 다음 제출 가능 시간: 2025-08-03 00:36:45 이후
15.354718923568726
